This code is a kaggle competition Bagofwordsmeetbagofpopcorn to predict the sentiment of novel movie reviews given a set of unlabeled and labeled movie reviews


In [ ]:
import nltk # natural language processing
from nltk.stem import PorterStemmer #stemming words
from nltk.tokenize import word_tokenize,sent_tokenize # tokenizing into words and sentences
from nltk.corpus import stopwords #stopwords in english
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("/home/dcb/Downloads/all"))


In [2]:
lbl_train = pd.read_csv('/home/dcb/Downloads/all/labeledTrainData.tsv',  sep= '\t', header = 0)
test_d =    pd.read_csv('/home/dcb/Downloads/all/testData.tsv',  sep= '\t', header = 0)
lbl_train.size,test_d.size

(75000, 50000)

In [3]:
total=lbl_train['review'].size + test_d["review"].size
total

50000

In [4]:
lbl_train.head(10)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
5,8196_8,1,I dont know why people think this is such a ba...
6,7166_2,0,"This movie could have been very good, but come..."
7,10633_1,0,I watched this video at a friend's house. I'm ...
8,319_1,0,"A friend of mine bought this film for £1, and ..."
9,8713_10,1,<br /><br />This movie is full of references. ...


the next cell is for preprocessing the text data.
1. removing html tags
2. using regex to remove specialcharacters
3. converting to lower case letters which makes the work easy for system
4. removing the less important words from text i.e stopwords and considering only nouns,adjectives which are useful in training the model

In [5]:
from bs4 import BeautifulSoup
import re
def raw_rev(rev):
    r_tag = BeautifulSoup(rev).get_text()                  ###removing html tags
    reg = re.sub("[^a-z/A-Z ]"," ",r_tag)                  ###excluding special characters
    normal = reg.lower()                                   ##normalizing text
    token = word_tokenize(normal)                          #####tokenizng
    stop_words = stopwords.words("english")                ### considering stopwords in english
    fi = []                                                  ###assigned a list
    [fi.append(w) for w in token if w not in stop_words]   ###creating a list excluding stopwords
    t =" ".join(fi)                                   ####joining words bac
    return (t)

In [6]:
#############filtered text
#import time
#start = time.time()
clean_train = []
clean_test = []
for i in range (0,lbl_train["review"].size):
     clean_train.append(raw_rev(lbl_train["review"][i]))
for j in range(0,test_d["review"].size):
     clean_test.append(raw_rev(test_d["review"][j]))
#end = end.time()
#print(start-end)

/home/dcb/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/dcb/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [10]:
clean_train[1]

'classic war worlds timothy hines entertaining film obviously goes great effort lengths faithfully recreate h g wells classic book mr hines succeeds watched film appreciated fact standard predictable hollywood fare comes every year e g spielberg version tom cruise slightest resemblance book obviously everyone looks different things movie envision amateur critics look criticize everything others rate movie important bases like entertained people never agree critics enjoyed effort mr hines put faithful h g wells classic novel found entertaining made easy overlook critics perceive shortcomings'

In [15]:
type(clean_train)

list

the next cell is to convert the text to numbers to feed to systemt to train model
CountVectorizer convert the words into tokens and a number is assigned 
This numbers are further converted to arrays

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                      stop_words = None, max_features = 5000)
tdf = vec.fit_transform(clean_train)
tdf = tdf.toarray()
tedf = vec.fit_transform(clean_test)
tedf = tedf.toarray()
tedf

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [34]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer().fit(tdf)
mes_tfidf = tfidf.transform(tdf)
test_tfidf = tfidf.transform(tedf)

the next cell is to use random classifier model to train data

In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

import time
start = time.time()
forest = RandomForestClassifier(n_estimators = 50)
forest = forest.fit(mes_tfidf,lbl_train["sentiment"])
result = forest.predict(test_tfidf)
op = pd.DataFrame(data= {"id":test_d["id"], "sentiment": result})
end = time.time()
print(start-end)    
#op.to_csv("/home/dcd/Downloads/all/ans.csv", index = False, quoting =3)

-25.338590621948242


In [42]:
op.head(20)


,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,1
3,7186_2,1
4,12128_7,0
5,2913_8,0
6,4396_1,1
7,395_2,0
8,10616_1,0
9,9074_9,1


In [47]:
print(forest.score(test_tfidf,lbl_train.sentiment)*100)

50.44
